In [ ]:
!nvidia-smi

Sat Jun 24 12:11:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-24 12:11:10--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-24 12:11:10 (22.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import *
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import random
import os
import io
from sklearn.model_selection import *
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.pipeline import *
from sklearn.metrics import *
from google.colab import *
import tensorflow_hub

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-06-24 12:11:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 142.251.111.128, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-06-24 12:11:16 (87.9 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data('nlp_getting_started.zip')

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 100)
train_df_shuffled.head()

,id,keyword,location,text,target
3999,5680,floods,Estados Unidos,Typhoon Soudelor approaches after 7 killed 2 m...,1
1849,2659,crush,EastAtlanta ??#WestGeorgia'18,WCE I can't even lie even tho I can't stand he...,0
5254,7514,oil%20spill,NaN,SYD traffic HAZARD Oil spill - BANKSTOWN Stace...,1
6645,9519,terrorist,Niall's arms,@ShipsXAnchors IDEK HOW IS THERE PEOPLE WHO A...,0
6461,9244,sunk,NaN,literally just sunk in we got backty school in...,0


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
random_index = random.randint(0, len(train_df) - 5)
for row in train_df_shuffled[['text', 'target']][random_index : random_index + 5].itertuples():
  _, text, target = row
  print(f'Target: {target}', '(Real Disaster)' if target > 0 else '(No Real Disaster)', f'\nText: {text}\n')

Target: 0 (No Real Disaster) 
Text: #Tampa: Super Freestyle Explosion Live in Concert at Amalie Arena - Sep  19
? Ticket Info: http://t.co/ooGotO76uZ

Target: 0 (No Real Disaster) 
Text: We destroyed the #Zimmerman fan club on Twitter @rzimmermanjr and we obliterated Renewsit reduced her to a sock acc. http://t.co/ybshleIE3p

Target: 1 (Real Disaster) 
Text: Is it seclusion when a class is evacuated and a child is left alone in the class to force compliance?  #MoreVoices

Target: 0 (No Real Disaster) 
Text: A random hole just broke out in the street ?? http://t.co/dWU8QqYs0v

Target: 1 (Real Disaster) 
Text: SoloQuiero Maryland mansion fire that killed 6 caused by damaged plug under Christmas tree report says - Into the flames: Firefighte...



In [ ]:
train_tweets, validation_tweets, train_labels, validation_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                                                            train_df_shuffled['target'].to_numpy(),
                                                                                                            test_size = 0.2,
                                                                                                            random_state = 100)

In [ ]:
len(train_tweets), len(train_labels), len(validation_tweets), len(validation_labels)

(6090, 6090, 1523, 1523)

In [ ]:
train_tweets[:10], train_labels[:10]

(array(['RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG',
        '@snapharmony : Bells toll in Hiroshima as Japan marks 70 years since atomic bombing http://t.co/yPvvqZ8jzt',
        'Very glad that you got obliterated in X Men First Class. You fully deserved it after all @kevinbacon soz',
        "NIggas playing in the thunderstorm.. 'HOPE THE LORR BLESS EM' ????",
        "witnessed my very first demolition derby today psa that my excitement has not worn down and it's been over 3 hours",
        'I liked a @YouTube video from @skippy6gaming http://t.co/MXhrextrkh Minecraft PS4 - 3 X 2 LAVA DOOR - How To - Tutorial ( PS3 / XBOX',
        'Do you ever just want to obliterate an entire species off the face of the earth? I vote for mosquitoes',
        '@tiggr_ why only Squad Obliteration?',
        'airplane crashes on house in Colombia 12 people die in accident https://t.co/ZhJlfLBHZL',
        '@ego_resolution im glad. My gay can heal 1

In [ ]:
text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens = None,
                                                                              standardize = 'lower_and_strip_punctuation',
                                                                              split = 'whitespace',
                                                                              ngrams = None,
                                                                              output_mode = 'int',
                                                                              output_sequence_length = None,
                                                                              pad_to_max_tokens = False)

In [ ]:
round(sum([len(i.split()) for i in train_tweets]) / len(train_tweets))

15

In [ ]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens = max_vocab_length,
                                                                               output_mode = 'int',
                                                                               output_sequence_length = max_length)


In [ ]:
text_vectorizer.adapt(train_tweets)

In [ ]:
sample_tweet = 'There is a earthquake here'
text_vectorizer([sample_tweet])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 75,   9,   3, 245, 139,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_tweet = random.choice(train_tweets)
print(f'Original Tweet:\n{random_tweet}\nVectorized Tweet:\n{text_vectorizer([random_tweet])}')

Original Tweet:
LZK issues Severe Thunderstorm Warning [wind: 60 MPH hail: 0.75 IN] for Sharp [AR] till 8:15 PM CDT  http://t.co/AUXSMdG1uN #WX
Vectorized Tweet:
[[   1  570  314  266  305  383  868  954  396 5886    4   10 2073 1775
  1004]]


In [ ]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [ ]:
text_embedder = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                          output_dim = 128,
                                          input_length = max_length)

In [ ]:
random_tweet = random.choice(train_tweets)
vectorized_tweet = text_vectorizer([random_tweet])
print(f'Original Tweet:\n{random_tweet}\nEmbedded Tweet:\n{text_embedder(vectorized_tweet)}')

Original Tweet:
i just drove with both my parents in the car lmao that was a panic attack waiting to happen
Embedded Tweet:
[[[-0.01174079 -0.01243784 -0.00907236 ... -0.00478907  0.04856468
    0.04825098]
  [-0.00869001 -0.04137974 -0.03743194 ... -0.01724573 -0.00135703
    0.01595241]
  [-0.03572951  0.03042972  0.04110415 ...  0.03340938 -0.02752266
   -0.03237523]
  ...
  [ 0.02511192 -0.01718807  0.01238508 ...  0.0160302   0.03979189
   -0.03043486]
  [-0.00351981  0.02062218  0.0346987  ...  0.02568015 -0.01572146
    0.04151127]
  [ 0.03526357 -0.00552316 -0.02086189 ...  0.02800925  0.04864252
    0.02520961]]]


In [ ]:
baseline_model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

baseline_model.fit(train_tweets,
            train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_model.score(validation_tweets,
              validation_labels)

0.7964543663821405

In [ ]:
baseline_preds = baseline_model.predict(validation_tweets)

In [ ]:
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model
  """
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_fscore, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'fscore': model_fscore}
  return model_results


In [ ]:
baseline_results = calculate_results(y_true = validation_labels,
                                     y_pred = baseline_preds)

In [ ]:
baseline_results

{'accuracy': 79.64543663821405,
 'precision': 0.8155429734561553,
 'recall': 0.7964543663821405,
 'fscore': 0.7893861889904019}

In [ ]:
SAVE_DIR = 'model_logs'

In [ ]:
inputs = tf.keras.layers.Input(shape= (1,),
                               dtype = tf.string)
x = text_vectorizer(inputs)
x = text_embedder(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name = 'model_1')

In [ ]:
model_1.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                metrics = ['accuracy'])

In [ ]:
model_1_history = model_1.fit(x = train_tweets,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_1')])

Saving TensorBoard log files to: model_logs/model_1/20230624-121121
Epoch 1/5
191/191 [==============================] - 37s 159ms/step - loss: 0.6219 - accuracy: 0.6772 - val_loss: 0.5485 - val_accuracy: 0.7472
Epoch 2/5
191/191 [==============================] - 6s 33ms/step - loss: 0.4571 - accuracy: 0.8149 - val_loss: 0.4691 - val_accuracy: 0.7945
Epoch 3/5
191/191 [==============================] - 2s 13ms/step - loss: 0.3568 - accuracy: 0.8594 - val_loss: 0.4479 - val_accuracy: 0.8109
Epoch 4/5
191/191 [==============================] - 2s 10ms/step - loss: 0.2891 - accuracy: 0.8878 - val_loss: 0.4498 - val_accuracy: 0.8096
Epoch 5/5
191/191 [==============================] - 1s 6ms/step - loss: 0.2385 - accuracy: 0.9133 - val_loss: 0.4684 - val_accuracy: 0.8056


In [ ]:
model_1.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 0s 3ms/step - loss: 0.4684 - accuracy: 0.8056


[0.4683661162853241, 0.8056467771530151]

In [ ]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1.predict(validation_tweets)))

48/48 [==============================] - 0s 2ms/step


In [ ]:
model_1_preds

<tf.Tensor: shape=(1523,), dtype=float32, numpy=array([1., 0., 0., ..., 1., 0., 0.], dtype=float32)>

In [ ]:
model_1_results = calculate_results(y_true = validation_labels,
                                    y_pred = model_1_preds)
model_1_results

{'accuracy': 80.56467498358502,
 'precision': 0.8126171780545436,
 'recall': 0.8056467498358503,
 'fscore': 0.8021026931111223}

In [ ]:
embedding_weights = model_1.get_layer('embedding').get_weights()[0]
embedding_weights

array([[-0.05156781,  0.01316283, -0.0489728 , ..., -0.05941593,
        -0.03070623, -0.02302383],
       [-0.01106765, -0.00160053,  0.02101578, ...,  0.0120591 ,
         0.0351616 , -0.0534285 ],
       [-0.01181642, -0.06204225, -0.00260624, ..., -0.05639545,
         0.03056618, -0.05616553],
       ...,
       [ 0.09967122,  0.05411277,  0.11321081, ...,  0.02916493,
         0.05306625,  0.04816388],
       [-0.07707983, -0.02481085, -0.05566128, ..., -0.02886245,
        -0.08023435, -0.00857647],
       [ 0.06559151,  0.07841047,  0.04301428, ...,  0.0564247 ,
         0.03993163,  0.06538331]], dtype=float32)

In [ ]:
vectors = io.open('vectors.tsv', 'w', encoding = 'utf-8')
metadata = io.open('metadata.tsv', 'w', encoding = 'utf-8')
for index, word in enumerate(vocab):
  if index == 0:
    continue
  vector = embedding_weights[index]
  vectors.write('\t'.join([str(x) for x in vector]) + '\n')
  metadata.write(word + '\n')
vectors.close()
metadata.close()

In [ ]:
inputs = tf.keras.layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = text_embedder(x)
x = tf.keras.layers.LSTM(64)(x)
outputs =  tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2')

In [ ]:
model_2.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])

In [ ]:
model_2_history = model_2.fit(x = train_tweets,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_2')])

Saving TensorBoard log files to: model_logs/model_2/20230624-121213
Epoch 1/5
191/191 [==============================] - 24s 102ms/step - loss: 0.2373 - accuracy: 0.9141 - val_loss: 0.5362 - val_accuracy: 0.7886
Epoch 2/5
191/191 [==============================] - 5s 26ms/step - loss: 0.1562 - accuracy: 0.9429 - val_loss: 0.6112 - val_accuracy: 0.7886
Epoch 3/5
191/191 [==============================] - 3s 15ms/step - loss: 0.1199 - accuracy: 0.9575 - val_loss: 0.7858 - val_accuracy: 0.7695
Epoch 4/5
191/191 [==============================] - 2s 9ms/step - loss: 0.0873 - accuracy: 0.9667 - val_loss: 0.8006 - val_accuracy: 0.7715
Epoch 5/5
191/191 [==============================] - 2s 9ms/step - loss: 0.0691 - accuracy: 0.9724 - val_loss: 0.9661 - val_accuracy: 0.7669


In [ ]:
model_2.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 0s 6ms/step - loss: 0.9661 - accuracy: 0.7669


[0.9661161303520203, 0.7669073939323425]

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2.predict(validation_tweets)))

48/48 [==============================] - 1s 4ms/step


In [ ]:
model_2_results = calculate_results(y_true = validation_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy': 76.69074195666448,
 'precision': 0.7671393394468594,
 'recall': 0.7669074195666448,
 'fscore': 0.7651173317575343}

In [ ]:
inputs = tf.keras.layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = text_embedder(x)
x = tf.keras.layers.GRU(64)(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3')

In [ ]:
model_3.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])


In [ ]:
model_3_history = model_3.fit(x = train_tweets,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_3')])

Saving TensorBoard log files to: model_logs/model_3/20230624-121258
Epoch 1/5
191/191 [==============================] - 14s 60ms/step - loss: 0.1225 - accuracy: 0.9542 - val_loss: 0.9980 - val_accuracy: 0.7715
Epoch 2/5
191/191 [==============================] - 3s 15ms/step - loss: 0.0697 - accuracy: 0.9713 - val_loss: 0.9757 - val_accuracy: 0.7669
Epoch 3/5
191/191 [==============================] - 3s 17ms/step - loss: 0.0676 - accuracy: 0.9724 - val_loss: 1.0415 - val_accuracy: 0.7702
Epoch 4/5
191/191 [==============================] - 2s 10ms/step - loss: 0.0467 - accuracy: 0.9777 - val_loss: 1.2217 - val_accuracy: 0.7505
Epoch 5/5
191/191 [==============================] - 1s 8ms/step - loss: 0.0393 - accuracy: 0.9800 - val_loss: 1.3163 - val_accuracy: 0.7452


In [ ]:
model_3.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 0s 6ms/step - loss: 1.3163 - accuracy: 0.7452


[1.3162649869918823, 0.7452396750450134]

In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3.predict(validation_tweets)))

48/48 [==============================] - 1s 5ms/step


In [ ]:
model_3_results = calculate_results(y_true = validation_labels,
                                    y_pred = model_3_preds)
model_3_results

{'accuracy': 74.52396585686147,
 'precision': 0.7445393237577126,
 'recall': 0.7452396585686146,
 'fscore': 0.744138961595224}

In [ ]:
inputs = tf.keras.layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = text_embedder(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = 'model_4')

In [ ]:
model_4.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])

In [ ]:
model_4_history = model_4.fit(x = train_tweets,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_4')])

Saving TensorBoard log files to: model_logs/model_4/20230624-121344
Epoch 1/5
191/191 [==============================] - 22s 75ms/step - loss: 0.0986 - accuracy: 0.9660 - val_loss: 0.9420 - val_accuracy: 0.7617
Epoch 2/5
191/191 [==============================] - 3s 14ms/step - loss: 0.0496 - accuracy: 0.9782 - val_loss: 1.4115 - val_accuracy: 0.7380
Epoch 3/5
191/191 [==============================] - 5s 24ms/step - loss: 0.0458 - accuracy: 0.9805 - val_loss: 1.3997 - val_accuracy: 0.7163
Epoch 4/5
191/191 [==============================] - 4s 23ms/step - loss: 0.0410 - accuracy: 0.9783 - val_loss: 1.1339 - val_accuracy: 0.7400
Epoch 5/5
191/191 [==============================] - 3s 15ms/step - loss: 0.0362 - accuracy: 0.9806 - val_loss: 1.4038 - val_accuracy: 0.7433


In [ ]:
model_4.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 0s 4ms/step - loss: 1.4038 - accuracy: 0.7433


[1.4037569761276245, 0.7432698607444763]

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(validation_tweets)))

48/48 [==============================] - 1s 3ms/step


In [ ]:
model_4_results = calculate_results(y_true = validation_labels,
                                    y_pred = model_4_preds)
model_4_results

{'accuracy': 74.32698621142481,
 'precision': 0.7425647100438444,
 'recall': 0.7432698621142482,
 'fscore': 0.7425649144392631}

In [ ]:
inputs = tf.keras.layers.Input(shape = (1, ), dtype = tf.string)
x =  text_vectorizer(inputs)
x = text_embedder(x)
x = tf.keras.layers.Conv1D(64, 7, 1, 'valid', activation = 'relu')(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name = 'model_5')

In [ ]:
model_5.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])

In [ ]:
model_5_history = model_5.fit(x = train_tweets,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_5')])

Saving TensorBoard log files to: model_logs/model_5/20230624-121432
Epoch 1/5
191/191 [==============================] - 27s 102ms/step - loss: 0.0918 - accuracy: 0.9652 - val_loss: 1.0532 - val_accuracy: 0.7360
Epoch 2/5
191/191 [==============================] - 4s 22ms/step - loss: 0.0487 - accuracy: 0.9810 - val_loss: 1.4149 - val_accuracy: 0.7380
Epoch 3/5
191/191 [==============================] - 5s 29ms/step - loss: 0.0375 - accuracy: 0.9829 - val_loss: 1.6815 - val_accuracy: 0.7479
Epoch 4/5
191/191 [==============================] - 2s 13ms/step - loss: 0.0320 - accuracy: 0.9839 - val_loss: 1.6976 - val_accuracy: 0.7308
Epoch 5/5
191/191 [==============================] - 1s 8ms/step - loss: 0.0282 - accuracy: 0.9844 - val_loss: 2.0323 - val_accuracy: 0.7452


In [ ]:
model_5.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 0s 3ms/step - loss: 2.0323 - accuracy: 0.7452


[2.0323221683502197, 0.7452396750450134]

In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5.predict(validation_tweets)))

48/48 [==============================] - 0s 2ms/step


In [ ]:
model_5_results = calculate_results(y_true = validation_labels, y_pred = model_5_preds)
model_5_results

{'accuracy': 74.52396585686147,
 'precision': 0.7447474746403292,
 'recall': 0.7452396585686146,
 'fscore': 0.7436527693662207}

In [ ]:
use_embedder = tensorflow_hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = 'universal_sentence_encoder')

In [ ]:
model_6 = tf.keras.Sequential([
    use_embedder,
    tf.keras.layers.Dense(64, activation = 'relu', name = 'dense_layer'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output_layer')
], name = 'model_6')

In [ ]:
model_6.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])

In [ ]:
model_6_history = model_6.fit(x = train_tweets,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_6')])

Saving TensorBoard log files to: model_logs/model_6/20230624-122427
Epoch 1/5
191/191 [==============================] - 5s 16ms/step - loss: 0.4650 - accuracy: 0.7936 - val_loss: 0.4382 - val_accuracy: 0.7925
Epoch 2/5
191/191 [==============================] - 3s 13ms/step - loss: 0.4015 - accuracy: 0.8245 - val_loss: 0.4328 - val_accuracy: 0.8017
Epoch 3/5
191/191 [==============================] - 3s 14ms/step - loss: 0.3853 - accuracy: 0.8323 - val_loss: 0.4318 - val_accuracy: 0.8056
Epoch 4/5
191/191 [==============================] - 4s 19ms/step - loss: 0.3666 - accuracy: 0.8415 - val_loss: 0.4426 - val_accuracy: 0.8043
Epoch 5/5
191/191 [==============================] - 3s 13ms/step - loss: 0.3503 - accuracy: 0.8511 - val_loss: 0.4364 - val_accuracy: 0.8168


In [ ]:
model_6.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 0s 9ms/step - loss: 0.4364 - accuracy: 0.8168


[0.43638288974761963, 0.8168089389801025]

In [ ]:
model_6_preds = tf.squeeze(tf.round(model_6.predict(validation_tweets)))

48/48 [==============================] - 1s 13ms/step


In [ ]:
model_6_results = calculate_results(y_true = validation_labels, y_pred = model_6_preds)
model_6_results

{'accuracy': 81.68089297439265,
 'precision': 0.8245490005854276,
 'recall': 0.8168089297439265,
 'fscore': 0.8134319103911938}

In [ ]:
train_data_10_percent = train_df_shuffled[['text', 'target']].sample(frac = 0.1, random_state = 100)
train_tweets_10_percent = train_data_10_percent['text'].to_list()
train_labels_10_percent = train_data_10_percent['target'].to_list()
len(train_tweets_10_percent), len(train_labels_10_percent)

(761, 761)

In [ ]:
train_data_10_percent['target'].value_counts()

0    413
1    348
Name: target, dtype: int64

In [ ]:
model_7 = tf.keras.Sequential([
    use_embedder,
    tf.keras.layers.Dense(64, activation = 'relu', name = 'dense_layer'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output_layer')
])

In [ ]:
model_7.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])

In [ ]:
model_7_history = model_7.fit(x = train_tweets_10_percent,
                              y = train_labels_10_percent,
                              epochs = 10,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_7')])

Saving TensorBoard log files to: model_logs/model_7/20230624-124245
Epoch 1/10
24/24 [==============================] - 2s 78ms/step - loss: 0.1159 - accuracy: 0.9763 - val_loss: 0.4002 - val_accuracy: 0.8733
Epoch 2/10
24/24 [==============================] - 1s 32ms/step - loss: 0.1006 - accuracy: 0.9803 - val_loss: 0.4152 - val_accuracy: 0.8667
Epoch 3/10
24/24 [==============================] - 1s 39ms/step - loss: 0.0911 - accuracy: 0.9855 - val_loss: 0.4138 - val_accuracy: 0.8720
Epoch 4/10
24/24 [==============================] - 1s 40ms/step - loss: 0.0812 - accuracy: 0.9855 - val_loss: 0.4223 - val_accuracy: 0.8726
Epoch 5/10
24/24 [==============================] - 1s 49ms/step - loss: 0.0701 - accuracy: 0.9895 - val_loss: 0.4256 - val_accuracy: 0.8739
Epoch 6/10
24/24 [==============================] - 1s 47ms/step - loss: 0.0622 - accuracy: 0.9882 - val_loss: 0.4371 - val_accuracy: 0.8759
Epoch 7/10
24/24 [==============================] - 1s 30ms/step - loss: 0.0571 - accu

In [ ]:
model_7.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 1s 15ms/step - loss: 0.4644 - accuracy: 0.8766


[0.4644007682800293, 0.8765594363212585]

In [ ]:
model_7_preds = tf.squeeze(tf.round(model_7.predict(validation_tweets)))

48/48 [==============================] - 2s 19ms/step


In [ ]:
model_7_results = calculate_results(y_true = validation_labels, y_pred = model_7_preds)
model_7_results

{'accuracy': 87.65594221930401,
 'precision': 0.8764660943107158,
 'recall': 0.8765594221930401,
 'fscore': 0.8764883401887749}

In [ ]:
train_data_10_percent_split = int(0.1 * len(train_tweets))
train_tweets_10_percent = train_tweets[:train_data_10_percent_split]
train_labels_10_percent = train_labels[:train_data_10_percent_split]

In [ ]:
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    347
1    262
dtype: int64

In [ ]:
model_8 = tf.keras.models.clone_model(model_7)

In [ ]:
model_8.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003),
                metrics = ['accuracy'])

In [ ]:
model_8_history = model_8.fit(x = train_tweets_10_percent,
                              y = train_labels_10_percent,
                              epochs = 10,
                              validation_data = (validation_tweets, validation_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'model_7')])

Saving TensorBoard log files to: model_logs/model_7/20230624-131116
Epoch 1/10
20/20 [==============================] - 1s 45ms/step - loss: 0.3076 - accuracy: 0.8703 - val_loss: 0.5267 - val_accuracy: 0.7400
Epoch 2/10
20/20 [==============================] - 1s 36ms/step - loss: 0.2893 - accuracy: 0.8703 - val_loss: 0.5362 - val_accuracy: 0.7387
Epoch 3/10
20/20 [==============================] - 1s 36ms/step - loss: 0.2725 - accuracy: 0.8768 - val_loss: 0.5432 - val_accuracy: 0.7433
Epoch 4/10
20/20 [==============================] - 2s 85ms/step - loss: 0.2607 - accuracy: 0.8818 - val_loss: 0.5435 - val_accuracy: 0.7505
Epoch 5/10
20/20 [==============================] - 1s 55ms/step - loss: 0.2424 - accuracy: 0.8900 - val_loss: 0.5518 - val_accuracy: 0.7459
Epoch 6/10
20/20 [==============================] - 1s 45ms/step - loss: 0.2374 - accuracy: 0.8933 - val_loss: 0.5654 - val_accuracy: 0.7426
Epoch 7/10
20/20 [==============================] - 1s 38ms/step - loss: 0.2172 - accu

In [ ]:
model_8.evaluate(validation_tweets, validation_labels)

48/48 [==============================] - 1s 18ms/step - loss: 0.6215 - accuracy: 0.7426


[0.6214892268180847, 0.7426132559776306]

In [ ]:
model_8_preds = tf.squeeze(tf.round(model_8.predict(validation_tweets)))

48/48 [==============================] - 1s 18ms/step


In [ ]:
model_8_results = calculate_results(y_true = validation_labels, y_pred = model_8_preds)
model_8_results

{'accuracy': 74.26132632961261,
 'precision': 0.7430953030832349,
 'recall': 0.7426132632961261,
 'fscore': 0.7428081086920291}

In [ ]:
all_models_results = pd.DataFrame({'naive-bayes': baseline_results,
                                   'dense': model_1_results,
                                   'lstm': model_2_results,
                                   'gru': model_3_results,
                                   'bidirectional': model_4_results,
                                   'conv1d': model_5_results,
                                   'use': model_6_results,
                                   'use_10_percent_leak': model_7_results,
                                   'use_10_percent': model_8_results,
                                   })

all_models_results = all_models_results.transpose()
all_models_results = all_models_results.reset_index()
all_models_results.rename(columns={'index':'model'}, inplace = True)

In [ ]:
all_models_results

,model,accuracy,precision,recall,fscore
0,naive-bayes,79.645437,0.815543,0.796454,0.789386
1,dense,80.564675,0.812617,0.805647,0.802103
2,lstm,76.690742,0.767139,0.766907,0.765117
3,gru,74.523966,0.744539,0.745240,0.744139
4,bidirectional,74.326986,0.742565,0.743270,0.742565
5,conv1d,74.523966,0.744747,0.745240,0.743653
6,use,81.680893,0.824549,0.816809,0.813432
7,use_10_percent_leak,87.655942,0.876466,0.876559,0.876488
8,use_10_percent,74.261326,0.743095,0.742613,0.742808


In [ ]:
all_model_results.plot(kind = 'bar', figsize = (10,7)).legend(bbox_to_anchor = (1.0, 1.0))

In [ ]:
!tensorboard dev upload --logdir ./model_logs/ --name 'NLP Modelling Experiments' --description 'Comparing different types of modelling architectures on the Kaggle Disaster Tweet dataset' --one-shot

In [ ]:
validation_df = pd.DataFrame({
    'tweet': validation_tweets,
    'label': validation_labels,
    'pred': model_6_preds,
    'pred_prob': tf.squeeze(model_6.predict(validation_twwets))
})

validaiton_df.head(10)

In [ ]:
most_wrong_predictions = validation_df[validation_df['label'] != ]validation_df['pred']].sort_values('pred_prob', ascending = False)
most_wrong_predicitons.head(10)

In [ ]:
model_6.save('use.h5')

In [ ]:
test_tweets = test_df['text'].to_list()
test_samples = random.sample(test_twwets, 10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6..predict(test_sample))
  pred = tf.round(pred_prob)
  print(f"Pred: {int(pred)}, Prob: {pred_prob}")
  print(f'Text:\n{test_sample}\n')

In [ ]:
def pred_timer(model, samples):
  """
  Returns time required for predictions and also for one prediction
  """
  start_time = time.perf_counter()
  model.predict(samples)
  end_time = time.perf_counter()
  total_time = end_time - start_time
  time_per_pred = total_time / len(samples)
  return total_time, time_per_pred

In [ ]:
model_6_total_pred_time, model_6_time_per_pred = pred_timer(model = model_6,
                                                            samples = validation_tweets)
model_6_total_pred_time, model_6_time_per_pred